<a href="https://colab.research.google.com/github/oecorrechag/Proyecto-Montreal-Energia/blob/master/LAB_precios_electricidad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Evolución de los Precios de la Electricidad
===

# Instrucciones

En el archivo `precios.zip`  se encuentran los archivos `Precio_Bolsa_Nacional_($kwh)_ano.xls` en formato de Microsoft Excel, los cuales contienen los precios históricos horarios de la electricidad para el mercado eléctrico Colombiano entre los años 1995 y 2017 en COL-PESOS/kWh. A partir de la información suministrada realice las siguientes actividades. 

**1.--** Lea los archivos y cree una tabla única concatenando la información para cada uno de los años. Debe transformar la tabla de tal forma que quede con las columnas `Fecha`, `Hora` y `Precio` (únicamente tres columnas).

**2.--** Compute e imprima el número de registros con datos faltantes.

**3.--** Compute e imprima el número de registros duplicados.

**4.--** Elimine los registros con datos duplicados o datos faltantes, e imprima la cantidad de registros que quedan (registros completos).

**5.--** Compute y grafique el precio primedio diario.

**6.--** Compute y grafique el precio máximo por mes.

**7.--** Compute y grafique el precio mínimo mensual.

**8.--** Haga un gráfico para comparar el precio máximo del mes (para cada mes) y el precio promedio mensual.

**9.--** Haga un histograma que muestre a que horas se produce el máximo precio diario para los días laborales.

**10.--** Haga un histograma que muestre a que horas se produce el máximo precio diario para los días sabado.

**11.--** Haga un histograma que muestre a que horas se produce el máximo precio diario para los días domingo.

**12.--** Imprima una tabla con la fecha y el valor más bajo por año del precio de bolsa.

**13.--** Haga una gráfica en que se muestre el precio promedio diario y el precio promedio mensual.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


---

# SOLUCION
**1.--** Lea los archivos y cree una tabla única concatenando la información para cada uno de los años. Debe transformar la tabla de tal forma que quede con las columnas `Fecha`, `Hora` y `Precio` (únicamente tres columnas).

In [0]:
# Paquetes.
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
import plotly.express as px
import plotly.graph_objects as go
import calendar
%matplotlib inline

In [0]:
# FUNCION PARA PRE-PROCESAMIENTO
# Esta funcion crea una copia de los datos.
# Verifica donde incian realmente los datos.
# Elimina las filas vacias.
# Con la funcion melt crea la estructura columnas 'Fecha', 'Hora' y 'Precio'.

def pre_processing(dfy):
    df = dfy.copy()
    df.columns = [f'col{i}' for i in range(len(df.columns))]

    # renombrar las columnas del dataframe
    columns = np.squeeze(df.loc[df[df.columns[0]] == 'Fecha'].values)
    columns = [str(col) for col in columns]
    columns = ['0.0' if col == '0' else col for col in columns]
    lindex = int(df.loc[df[df.columns[0]] == 'Fecha'].index.values) + 1
    df.columns = columns
    
    # eliminar las primeras filas inválidas
    df = df.drop(list(range(0, lindex)))
    
    # invierte una tabla de 2 dimensiones en una lista
    df = pd.melt(
        df,
        id_vars='Fecha', 
        value_vars=list(df.columns[1:25]),
        var_name='Hora', 
        value_name='Precio'
    )
    
    return df

In [0]:
# Ubicacion y carga de datos  
p = Path(r'/content/drive/My Drive/Ciencia de los datos/laboratorios-de-programacion-oecg/lab-analisis-exploratorio-de-datos-oecg/precios')  # carpeta donde estan los archivos xls o xlsx
dfs = [pd.read_excel(file) for file in p.glob('*.xls*')]

In [8]:
# Muestra el head 
dfs[0].head()

,Precio Bolsa Nacional ($/kWh) 2001,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Fecha,0.000000,1.000000,2.000000,3.000000,4.000000,5.000000,6.000000,7.000000,8.000000,9.000000,10.000000,11.000000,12.000000,13.000000,14.000000,15.000000,16.000000,17.000000,18.000000,19.000000,20.000000,21.000000,22.000000,23.000000
2,2001-01-01,53.727589,53.727589,50.727589,50.727589,48.727589,46.727589,46.727589,46.727589,46.727589,46.727589,46.727589,46.727589,46.727589,46.727589,46.727589,46.727589,46.727589,46.727589,53.727589,55.727589,55.727589,46.727589,48.727589,46.727589
3,2001-01-02,46.727589,46.727589,46.727589,46.727589,46.727589,46.727589,46.727589,48.727589,54.727589,54.727589,54.727589,54.727589,54.727589,54.727589,54.727589,54.727589,54.727589,54.727589,82.727589,82.707589,82.707589,54.727589,54.727589,52.727589
4,2001-01-03,46.727589,46.727589,46.727589,46.727589,46.727589,52.727589,54.727589,54.727589,54.727589,54.727589,54.727589,54.727589,54.727589,54.727589,54.727589,54.727589,54.727589,54.727589,65.727589,85.707589,75.727589,54.727589,54.727589,52.727589


In [0]:
# Preprocesar los datos 
dfs_processed = []
for i, year in enumerate(range(1995, 2019)):
    try:
        dfs_processed.append(pre_processing(dfs[i]))
    except:
        year = str(year)
        print(f'Error en: {year}')

In [0]:
# Procesar los datos: concatenar todos los df
df_concatenated = pd.concat(dfs_processed, axis=0).reset_index(drop=True)

In [11]:
# Verificacion de resultado
df_concatenated.head(10)
df_concatenated.tail(10)

,Fecha,Hora,Precio
201806,2002-12-22,23.0,62.5515
201807,2002-12-23,23.0,62.5015
201808,2002-12-24,23.0,62.4515
201809,2002-12-25,23.0,61.0465
201810,2002-12-26,23.0,60.9015
201811,2002-12-27,23.0,60.9515
201812,2002-12-28,23.0,61.0415
201813,2002-12-29,23.0,60.8515
201814,2002-12-30,23.0,59.0015
201815,2002-12-31,23.0,58.9615


In [13]:
df_concatenated.shape

(201816, 3)

Se tienen 201.816 registros, pero en ellos pueden existir datos faltantes y datos repetidos.  

**2.--** Compute e imprima el número de registros con datos faltantes.

In [14]:
df_concatenated.isnull().sum()

Fecha     216
Hora        0
Precio    238
dtype: int64

In [15]:
df_concatenated[df_concatenated['Fecha'].isnull()].shape
df_concatenated[df_concatenated['Precio'].isnull()].shape

(238, 3)

Se tienen 216 faltantes en la variable Fecha y 238 faltantes en la variable Precio, no se tiene claro si estos faltantes pertenecen a la mismo registro.

**3.--** Compute e imprima el número de registros duplicados.

In [16]:
df_concatenated.duplicated().sum()

1951

Se tienen 1951 registros duplicados, por ejemplo para el año 2007 

**4.--** Elimine los registros con datos duplicados o datos faltantes, e imprima la cantidad de registros que quedan (registros completos).

In [17]:
df = df_concatenated.dropna()
df = df.drop_duplicates()
df.shape

(199819, 3)

Al retirar los registros faltantes y los registros duplicados se tienen 199.819 registros validos.

**5.--** Compute y grafique el precio primedio diario.

In [0]:
# Aplicar formato a las variables 
df['Precio'] = df['Precio'].astype('float32')
df['Fecha']  = pd.to_datetime(df['Fecha'])

In [19]:
# Agrupar por dias y sacar el promedio por dias, agrupar por el dia 
graph = df.groupby('Fecha').mean()
graph = graph.reset_index()
graph.shape

(8326, 2)

In [22]:
df.groupby('Fecha').mean().reset_index()

,Fecha,Precio
0,1995-07-20,1.409435
1,1995-07-21,4.924333
2,1995-07-22,1.269500
3,1995-07-23,0.953083
4,1995-07-24,4.305917
...,...,...
8321,2018-05-01,71.666725
8322,2018-05-02,76.374443
8323,2018-05-03,71.148346
8324,2018-05-04,64.710114


In [21]:
# Grafica Fecha x Precio
fig = px.line(graph, x = 'Fecha', y = 'Precio',
              title = "Precio promedio diario (kws) desde 1996 hasta 2018",
              template = "plotly_dark")
fig.show()

En la figura anterior se observa un crecimiento constante del precio promedio diario (kws) hasta el año 2010, los siguientes años el crecimiento la variacion es mucho mayor. Adicionalmente existe un pico en el 2017, el cual se debe al fenomeno del niño.

**6.--** Compute y grafique el precio máximo por mes.

In [23]:
# Se crea una copia del dataframe df
dfM = df.copy()
# Aplicar formato a las variables 
dfM['Precio'] = dfM['Precio'].astype('float32')
dfM['Fecha']  = pd.to_datetime(dfM['Fecha'])
# Generar nueva variable Mes
dfM['Mes2'] = dfM['Fecha'].apply(lambda x: x.month)
dfM['Mes'] = dfM['Mes2'].apply(lambda x: calendar.month_name[x])
graph2 = dfM.groupby('Mes').max()[['Precio', 'Mes2']]
graph2 = graph2.reset_index()
graph2 = graph2.sort_values(by=['Mes2'])
graph2.head()

,Mes,Precio,Mes2
4,January,865.477112,1
3,February,869.442017,2
7,March,884.836670,3
0,April,807.500427,4
8,May,686.494568,5


In [24]:
# Grafica Mes x Precio
fig = px.line(graph2, x = 'Mes', y = 'Precio',
              title = "Precio maximo Mensual (kws)",
              template = "plotly_dark")
fig.show()

De la figura anterior se pede observar que el precio maximo se dio en el mes de Octubre.

**7.--** Compute y grafique el precio mínimo mensual.

In [0]:
# Se crea una copia del dataframe df
dfm = df.copy()
# Aplicar formato a las variables 
dfm['Precio'] = dfm['Precio'].astype('float32')
dfm['Fecha']  = pd.to_datetime(dfm['Fecha'])
# Generar nueva variable Mes
dfm['Mes2'] = dfm['Fecha'].apply(lambda x: x.month)
dfm['Mes'] = dfm['Mes2'].apply(lambda x: calendar.month_name[x])
graph3 = dfm.groupby('Mes').min()[['Precio', 'Mes2']]
graph3 = graph3.reset_index()
graph3 = graph3.sort_values(by=['Mes2'])

In [26]:
# Grafica Mes x Precio
fig = px.line(graph3, x = 'Mes', y = 'Precio',
              title = "Precio minimo mensual (kws)",
              template = "plotly_dark")
fig.show()

De la figura anterior se pede observar que el precio minimo se dio en los meses de Mayo, Junio, Julio, Agosto y Octubre.

**8.--** Haga un gráfico para comparar el precio máximo del mes (para cada mes) y el precio promedio mensual.

In [0]:
# Se crea una copia del dataframe df
dfme = df.copy()
# Aplicar formato a las variables 
dfme['Precio'] = dfme['Precio'].astype('float32')
dfme['Fecha']  = pd.to_datetime(dfme['Fecha'])
# Generar nueva variable Mes
dfme['Mes2'] = dfme['Fecha'].apply(lambda x: x.month)
dfme['Mes'] = dfme['Mes2'].apply(lambda x: calendar.month_name[x])
graph4 = dfme.groupby('Mes').mean()[['Precio', 'Mes2']]
graph4 = graph4.reset_index()
graph4 = graph4.sort_values(by=['Mes2'])

In [28]:
# Se crea una copia del dataframe df
graph2B = graph2.copy()
graph4B = graph4.copy()

# Se elimina la variable 'Mes' y 'Mes2'
graph2B = graph2B.drop(['Mes', 'Mes2'], axis=1)

# Renombra las columnas
graph2B.rename(columns={'Precio':'Precio_max'}, inplace=True)
graph4B.rename(columns={'Precio':'Precio_promedio'}, inplace=True)

# Juntar los df
df8 = pd.concat([graph2B, graph4B], axis=1)

# Re ordenar las columnas
df8 = df8[['Mes2', 'Mes', 'Precio_max', 'Precio_promedio']]
df8

,Mes2,Mes,Precio_max,Precio_promedio
4,1,January,865.477112,111.924492
3,2,February,869.442017,125.139290
7,3,March,884.836670,125.732941
0,4,April,807.500427,106.910423
8,5,May,686.494568,91.602249
6,6,June,474.110748,83.849152
5,7,July,445.329376,79.269066
1,8,August,441.242920,83.978668
11,9,September,1495.042969,107.400101
10,10,October,2821.520508,143.987610


In [29]:
fig = go.Figure()
fig.add_trace(go.Scatter(
                x = df8['Mes'],
                y = df8['Precio_max'],
                name = "Precio Maximo",
                line_color = 'deepskyblue',
                opacity = 0.8))

fig.add_trace(go.Scatter(
                x = df8['Mes'],
                y = df8['Precio_promedio'],
                name = "Precio promedio",
                line_color = 'dimgray',
                opacity = 0.8))

# Use date string to set xaxis range
fig.update_layout(title_text="Precio maximo y precio promedio (kws) ",
                 template = "plotly_dark")
fig.show()

**9.--** Haga un histograma que muestre a que horas se produce el máximo precio diario para los días laborales.

In [0]:
# Se crea una copia del dataframe df
df9 = df.copy()
# Se crea una variable nueva numero de dia
df9['Day'] = df9['Fecha'].apply(lambda x: x.isoweekday())
# tomar solo los dias laborales 1-6 (los lunes a sabado)
df9L = df9.loc[df9['Day'].isin(list(range(1, 6))),:]

In [36]:
# Agrupar y calcular el precio maximo de cada dia
dfPM = df9L.groupby('Fecha').max()[['Precio']]
# Cambiar nombre de la variable para evitar duplicar nombre al hacer el join
dfPM.columns = ['Precio Maximo']
dfPM = dfPM.reset_index()
dfPM.head()

,Fecha,Precio Maximo
0,1995-07-20,2.827
1,1995-07-21,16.670
2,1995-07-24,22.500
3,1995-07-25,1.500
4,1995-07-26,8.000


In [37]:
# Join para juntar df9L y dfPM por la fecha
df9PM = pd.merge(df9L, dfPM, on = ['Fecha'])
# Obtener el maximo del dia 
df9PM = df9PM[df9PM['Precio'] == df9PM['Precio Maximo']]
# Volver entero la variable 'Hora'
df9PM['Hora'] = df9PM['Hora'].apply(lambda x: int(float(x)))
# Se elimina la variable 'Precio'
df9PM = df9PM.drop(['Precio'], axis=1)
df9PM.head()

,Fecha,Hora,Day,Precio Maximo
19,2001-01-01,19,1,55.727589
20,2001-01-01,20,1,55.727589
42,2001-01-02,18,2,82.727592
67,2001-01-03,19,3,85.707588
82,2001-01-04,10,4,70.727592


In [38]:
fig = px.histogram(df9PM, x = "Hora",
                   title = "Precio maximo por horas para los dias laborales",
                   template = "plotly_dark")
fig.show()

De la figura anterior se observa que para los dias lunes a viernes, el precio maximo por hora se presenta entre las 6 y las 8, siendo las 7 de la noche la hora con el precio mayor.

**10.--** Haga un histograma que muestre a que horas se produce el máximo precio diario para los días sabado.

In [0]:
# Se crea una copia del dataframe df
df10 = df.copy()
# Se crea una variable nueva numero de dia
df10['Day'] = df10['Fecha'].apply(lambda x: x.isoweekday())
# tomar solo los dias laborales
df10L = df10.loc[df10['Day'] == 6,:]

In [40]:
# Agrupar y calcular el precio maximo de cada dia
dfPMS = df10L.groupby('Fecha').max()[['Precio']]
# Cambiar nombre de la variable para evitar duplicar nombre al hacer el join
dfPMS.columns = ['Precio Maximo']
dfPMS = dfPMS.reset_index()
dfPMS.head()

,Fecha,Precio Maximo
0,1995-07-22,2.500
1,1995-07-29,1.072
2,1995-08-05,2.558
3,1995-08-12,40.000
4,1995-08-19,2.000


In [41]:
# Join para juntar df10L y dfPMS por la fecha
df10PM = pd.merge(df10L, dfPMS, on = ['Fecha'])
# Obtener el maximo del dia 
df10PM = df10PM[df10PM['Precio'] == df10PM['Precio Maximo']]
# Volver entero la variable 'Hora'
df10PM['Hora'] = df10PM['Hora'].apply(lambda x: int(float(x)))
# Se elimina la variable 'Precio'
df10PM = df10PM.drop(['Precio'], axis=1)
df10PM.head()

,Fecha,Hora,Day,Precio Maximo
19,2001-01-06,19,6,85.727592
43,2001-01-13,19,6,100.627586
57,2001-01-20,9,6,99.727592
69,2001-01-20,21,6,99.727592
79,2001-01-27,7,6,99.727592


In [42]:
fig = px.histogram(df10PM, x = "Hora",
                   title = "Precio maximo por horas para el dia Sabado",
                   template = "plotly_dark")
fig.show()

De la figura anterior se observa que para el dia Sabado, el precio maximo por hora se presenta entre las 6 y las 8, siendo las 7 de la noche la hora con el precio mayor.

**11.--** Haga un histograma que muestre a que horas se produce el máximo precio diario para los días domingo.

In [0]:
# Se crea una copia del dataframe df
df11 = df.copy()
# Se crea una variable nueva numero de dia
df11['Day'] = df11['Fecha'].apply(lambda x: x.isoweekday())
# tomar solo los dias laborales
df11L = df11.loc[df11['Day'] == 7,:]

In [44]:
# Agrupar y calcular el precio maximo de cada dia
dfD = df11L.groupby('Fecha').max()[['Precio']]
# Cambiar nombre de la variable para evitar duplicar nombre al hacer el join
dfD.columns = ['Precio Maximo']
dfD = dfD.reset_index()
dfD.head()

,Fecha,Precio Maximo
0,1995-07-23,1.238
1,1995-07-30,1.073
2,1995-08-06,3.200
3,1995-08-13,40.000
4,1995-08-20,3.100


In [45]:
# Join para juntar df11L y dfD por la fecha
df11PM = pd.merge(df11L, dfD, on = ['Fecha'])
# Obtener el maximo del dia 
df11PM = df11PM[df11PM['Precio'] == df11PM['Precio Maximo']]
# Volver entero la variable 'Hora'
df11PM['Hora'] = df11PM['Hora'].apply(lambda x: int(float(x)))
# Se elimina la variable 'Precio'
df11PM = df11PM.drop(['Precio'], axis=1)
df11PM.head()

,Fecha,Hora,Day,Precio Maximo
18,2001-01-07,18,7,85.727592
19,2001-01-07,19,7,85.727592
43,2001-01-14,19,7,100.527588
69,2001-01-21,21,7,99.727592
90,2001-01-28,18,7,70.727592


In [46]:
fig = px.histogram(df11PM, x = "Hora",
                   title = "Precio maximo por horas para el dia Domingo",
                   template = "plotly_dark")
fig.show()

De la figura anterior se observa que para el dia Domingo, el precio maximo por hora se presenta entre las 6 y las 8, siendo las 7 de la noche la hora con el precio mayor.

**12.--** Imprima una tabla con la fecha y el valor más bajo por año del precio de bolsa.

In [0]:
# Se crea una copia del dataframe df
df12 = df.copy()
# Se crea una variable nueva numero de dia
df12['Year'] = df11['Fecha'].apply(lambda x: x.year)
# Agrupar y calcular el precio minimo de cada año
df12 = df12.groupby('Year').max()[['Fecha','Precio']]
df12 = df12.reset_index()
df12

,Year,Fecha,Precio
0,1995,1995-12-31,179.729996
1,1996,1996-12-31,122.674004
2,1997,1997-12-31,273.376404
3,1998,1998-12-31,236.025757
4,1999,1999-12-31,67.376617
5,2000,2000-12-31,96.137932
6,2001,2001-12-31,118.072762
7,2002,2002-12-31,138.547485
8,2003,2003-12-31,148.003601
9,2004,2004-12-31,189.902771


**13.--** Haga una gráfica en que se muestre el precio promedio diario y el precio promedio mensual.

In [47]:
# Se crea una copia del dataframe df
df13 = df.copy()
# se crea una variable dia y otra variable mes
df13['Dia'] = df13['Fecha'].apply(lambda x: x.day)
df13['Mes'] = df13['Fecha'].apply(lambda x: x.month)
df13.head()

,Fecha,Hora,Precio,Dia,Mes
0,2001-01-01,0.0,53.727589,1,1
1,2001-01-02,0.0,46.727589,2,1
2,2001-01-03,0.0,46.727589,3,1
3,2001-01-04,0.0,52.727589,4,1
4,2001-01-05,0.0,51.727589,5,1


In [49]:
# Agrupar y calcular el precio promedio de cada dia del mes
dfdia = df13.groupby('Dia').mean()['Precio']
dfdia = dfdia.reset_index()
dfdia.head()

,Dia,Precio
0,1,107.854332
1,2,108.913261
2,3,110.298927
3,4,111.968628
4,5,111.388786


In [50]:
# Agrupar y calcular el precio promedio de cada mes
dfmes = df13.groupby('Mes').mean()['Precio']
dfmes = dfmes.reset_index()
dfmes.head()

,Mes,Precio
0,1,111.924492
1,2,125.139290
2,3,125.732941
3,4,106.910423
4,5,91.602249


In [51]:
# Grafica Mes x Precio
fig = px.line(dfdia, x = 'Dia', y = 'Precio',
              title = "Precio promedio diario (kws)",
              template = "plotly_dark")
fig.show()

In [52]:
# Grafica Mes x Precio
fig = px.line(dfmes, x = 'Mes', y = 'Precio',
              title = "Precio promedio mensual (kws)",
              template = "plotly_dark")
fig.show()

In [53]:
# Se crea una copia del dataframe df
df14 = df.copy()
# se crea una variable mes por cada año
df14['Mensual'] = df14['Fecha'].apply(lambda x: x.strftime('%Y-%m'))
# Agrupar y calcular el precio promedio de cada mes de cada año
df14M = df14.groupby('Mensual').mean()['Precio']
df14M = df14M.reset_index()
# Renombra las columnas
df14M.rename(columns={'Mensual':'Fecha'}, inplace=True)
df14M.head()

,Fecha,Precio
0,1995-07,1.540655
1,1995-08,7.086462
2,1995-09,10.955819
3,1995-10,10.445442
4,1995-11,27.534782


In [54]:
# Grafica Fecha x Precio
fig = px.line(graph, x = 'Fecha', y = 'Precio',
              title = "Precio promedio diario (kws) desde 1996 hasta 2018",
              template = "plotly_dark")
fig.show()

In [55]:
# Grafica Fecha x Precio
fig = px.line(df14M, x = 'Fecha', y = 'Precio',
              title = "Precio promedio mensual (kws) desde 1996 hasta 2018",
              template = "plotly_dark")
fig.show()